# NSRR MrOS (Visit 1) Features extraction

https://sleepdata.org/datasets/mros/pages/equipment-mros1.md

https://sleepdata.org/datasets/mros/variables/pooveral

In [ ]:
import os
import yasa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf

root_dir = '/Volumes/NSRR/mros/'
eeg_dir = root_dir + 'polysomnography/edfs/visit1/'
hypno_dir = root_dir + 'polysomnography/annotations-events-profusion/visit1/'

desc_dir = root_dir + 'datasets/mros-visit1-dataset-0.4.0.csv'

usecols = ['nsrrid', 'visit', 'gender', 'vsage1', 'pooveral', 'hwbmi', 'gierace', 'poordi3']

df_subj = pd.read_csv(desc_dir, usecols=usecols)

# Rename columns
df_subj.rename(columns={'nsrrid': 'subj',
                        'vsage1': 'age',
                        'pooveral': 'overall',
                        'hwbmi': 'bmi',
                        'gierace': 'ethnicity',
                        'poordi3': 'ahi',
                      }, inplace=True)

# Keep only "Excellent" quality study
print(df_subj[df_subj['overall'] < 6].shape[0], 
      'subjects with bad PSG data quality will be removed.')
df_subj = df_subj[df_subj['overall'] >= 6]

# They should all be male in MrOS!
df_subj['male'] = (df_subj['gender'] == 2).astype(int)
df_subj['ethnicity'].replace({1: 'caucasian', 2: 'african', 3: 'other', 
                              4: 'hispanic', 5: 'other'}, inplace=True)

# Keep only first visit
df_subj = df_subj[df_subj['visit'] == 1]

# Convert to str
df_subj['subj'] = df_subj['subj'].apply(lambda x: str(x).zfill(4))
df_subj.set_index('subj', inplace=True)

# Export demographics to CSV file
df_subj['dataset'] = 'MROS'
df_subj.to_csv("/Volumes/JAWA/SHERLOCK/demo/demo_nsrr_mros.csv")

# Keep only a random subset of 600 subjects to avoid dataset imbalance
df_subj = df_subj.sample(n=600, replace=False, random_state=42)

print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

In [ ]:
df = []
include = ['C4', 'LOC', 'L Chin']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'mros-visit1-' + str(sub) + '.edf'
    hypno_file = hypno_dir + 'mros-visit1-' + str(sub) + '-profusion.xml'

    # LOAD EEG DATA
    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw = read_raw_edf(eeg_file, preload=True, 
                           exclude=np.setdiff1d(raw.info['ch_names'], include), 
                           verbose=0)
        # Skip subjects if channel were not found
        assert len(raw.ch_names) == len(include)
    except:
        continue
        
    # Resample and high-pass filter 
    raw.resample(sf, npad="auto")

    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # We keep up to 15 minutes before / after sleep
    start_to_firstsleep_min = np.nonzero(hypno)[0][0] / 2
    lastsleep = np.nonzero(hypno)[0][-1]
    lastsleep_to_end_min = (len(hypno) - lastsleep) / 2
    tmin, tmax = 0, None  # must be in seconds
    if start_to_firstsleep_min > 15:
        tmin = (start_to_firstsleep_min - 15) * 60
    if lastsleep_to_end_min > 15:
        tmax = lastsleep * 30 + 15 * 60
    # Crop!
    raw.crop(tmin, tmax)
    if tmax is None:
        hypno = hypno[int(tmin / 60 * 2):]
    else:
        hypno = hypno[int(tmin / 60 * 2):int(tmax / 60 * 2)]
    # Hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA
    # Hypnogram must include all stages
    if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
        print("- Not all stages are present.")
        continue
    # If the duration is not between 4 to 12 hours, skip subject
    if not(4 < n_epochs / 120 < 12):
        print("- Recording too short/long.")
        continue
    # Requires at least 5 min of each stage
    # if (stage_min < 5).any():
    #    print("- Not 5 min of each stage.")
    #    continue
       
    # EXTRACT FEATURES
    metadata = dict(age=df_subj.loc[sub, 'age'], male=df_subj.loc[sub, 'male'])
    sls = yasa.SleepStaging(raw, eeg_name=include[0], eog_name=include[1], 
                            emg_name=include[2], metadata=metadata)

    features = sls.get_features().reset_index()
    features['subj'] = sub
    features.set_index(['subj', 'epoch'], inplace=True)
    
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    df.append(features)

df = pd.concat(df)

In [ ]:
# Add dataset
df['dataset'] = 'mros'

# Convert to category
df['dataset'] = df['dataset'].astype('category')
df['stage'] = df['stage'].astype('category')

In [ ]:
# %stage
df['stage'].value_counts(normalize=True, sort=True)

In [ ]:
# Median value of the EEG IQR per stage
df.groupby('stage')['eeg_iqr'].median()

In [ ]:
# Number of unique nights in dataset
df.index.get_level_values(0).nunique()

In [ ]:
# Export
df.to_parquet("/Volumes/JAWA/SHERLOCK/features/features_nsrr_mros1.parquet")